In [71]:
import os
import pandas as pd
import re
import numpy as np
import collections
from operator import itemgetter
import pprint as pp
import re
import nltk
from nltk.corpus import stopwords
import nltk.data
import time


#data from http://times.cs.uiuc.edu/~wang296/Data/
files = os.listdir('./Review_Texts')
#print(os.listdir('./Review_Texts'))
print(len(files))

1759


In [72]:
cats = ['Rooms', 'Date', 'Location', 'Service', 'Business service', 'Author', 'Check in / front desk', 'No. Helpful', 'Cleanliness', 'Content', 'Value', 'No. Reader', 'Overall']
data = pd.DataFrame(columns=cats)

In [73]:
def addFileToData(filename, data):
    intColumns = ['No. Reader', 'No. Helpful', 'Cleanliness','Check in / front desk', 'Value', 'Overall', 'Service', 'Business service', 'Rooms', 'Location']
    characterThreshold = 60
    with open(filename, 'r') as content_file:
        content = content_file.read()
        
    reviews = content.split("\n\n")
    for r in reviews:
        thisReview = pd.Series([None]*len(cats), cats)
        splt = r.split("\n")
        for s in splt:
            for c in cats:
                if "<"+c+">" in s:
                    value = s.replace('<'+c+'>', '')
                    if c in intColumns:
                        value = int(value)
                    if value == -1: #we dont want -1 as this is going to mess up averaging, take np.nan
                        value = np.nan

                    if c == "Content":
                        value = value.lower()

                    thisReview[c] = value
                    
        if not thisReview["Content"] == None and len(thisReview["Content"]) > characterThreshold:
            #only add if theres content and its long enough
            data = data.append(thisReview, ignore_index=True)
    return data
        
data = addFileToData('./Review_Texts/hotel_100504.dat', data) 
data = addFileToData('./Review_Texts/hotel_72572.dat', data) 


In [4]:
print(data.shape)
data.head()

(199, 13)


,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Dec 23, 2008",5.0,5.0,NaN,selizabethm,5.0,NaN,5.0,wonderful time- even with the snow! what a gre...,4.0,NaN,5.0
1,4.0,"Nov 13, 2008",5.0,5.0,NaN,IndieLady,5.0,NaN,4.0,"lovely hotel, unique decor, friendly front des...",5.0,NaN,4.0
2,4.0,"Nov 11, 2008",3.0,NaN,4,Hilobb,5.0,NaN,4.0,"nice hotel, expensive parking we got a good de...",4.0,NaN,4.0
3,5.0,"Nov 4, 2008",5.0,5.0,5,Chianti_girl24,5.0,NaN,5.0,fabulous hotel location and service are great....,5.0,NaN,5.0
4,NaN,"Oct 18, 2008",NaN,NaN,NaN,hothearted,NaN,2,NaN,"loved the monaco! staff was amazing, with a sm...",NaN,2,5.0


In [6]:
#tokenizer to split sentences
from nltk.tokenize import RegexpTokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer2 = RegexpTokenizer(r'\w+')

#Stemmer defined for stemming words
from nltk.stem.porter import *
stemmer = PorterStemmer()

# -----------------------------------------------------------------------------------------
#                                    FEATURE EXTRACTION: MODEL RUN
# -----------------------------------------------------------------------------------------

In [166]:
from nltk import bigrams
from nltk.util import ngrams
from sklearn.feature_extraction import DictVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
import string
from string import * 
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split


#Stemmer defined for stemming words
from nltk.stem.porter import *
stemmer = PorterStemmer()


# print data['Content'][0]

#Function to clean reviews
def clean_string(review):
    sentence=review.lower()
    word_list=sentence.split()
    sentence=' '.join([i for i in word_list if i not in remove_list])
    translation=string.maketrans("","")
    clean_review = sentence.translate(translation,string.punctuation)
    return clean_review

# Clean all reviews of dataset and add to new list for feature creation
def Model_run(reviews_aspect_series,ratings_series):
    
    new_data_review=[]
    for i in reviews_aspect_series:
        new_data_review.append(clean_string(i))

#Obain unigram features (create a function for this)
    unigram_vectorizer = CountVectorizer(min_df=1,binary=False)
    corpus = new_data_review
    X = unigram_vectorizer.fit_transform(corpus)
    X_1=X.toarray()
    print len(X_1[0])

# Obtain Bi-unigram features (create a function for this)
    bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)
    X_2 = bigram_vectorizer.fit_transform(corpus).toarray()
    print len(X_2[0])

# Get complete feature set
    X=np.concatenate((X_1,X_2),axis=1)
    print len(X)
    print len(X[0])
#Use tf-idf transformer on the unigram/bigram features (create function here)
    transformer = TfidfTransformer(smooth_idf=False)
    tfidf = transformer.fit_transform(X_1)
    tfidf.toarray()

    Y=ratings_series
    np.array(Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
# clf.fit(tfidf[:160].toarray(), Y[:160])
# clf.score(X_test, y_test)

    params=clf.get_params(deep=True)
    print params

    sum1=0
    count=0
    for i in xrange(0,len(X_test)):
        a=clf.predict(X_test[i])
        b=np.array(y_test)[i]
        if a[0]== b:
            ind=1
        else:
            ind=0
        sum1=sum1+ind
        count=count+1
    return (sum1*1.0/count)

Model_run(data['Content'],data['Overall'])

C:\Users\Vinayak11\Anaconda\lib\site-packages\ipykernel\__main__.py:23: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


3776
17877
199
21653
{'priors': None}


C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  Deprec

0.5

In [161]:
#Get frequency distribution of cleaned words and select one with high frequency for unigrams and bigrams
new_data_review=[]
for i in data['Content']:
    new_data_review.append(clean_string(i))

word_list=[]
for i in new_data_review:
    temp=i.split()
    word_list.extend(temp)

word_freq = [word_list.count(w) for w in word_list]
freq_dict=dict(zip(word_list,word_freq))
sorted_list = sorted(freq_dict.items(), key=operator.itemgetter(1))

# take words with frequency > 1
lowfreq_word_list=[]
for pair in sorted_list:
    if pair[1]<10:
        lowfreq_word_list.append(pair[0])
    else:
        continue
#remove low frequency words from new_data_review

reviews_final=[]
for i in new_data_review:
    a=i.split()
    for j in a:
        if j in lowfreq_word_list:
            a.remove(j)
    sentence=' '.join([k for k in a])
    reviews_final.append(sentence)
print reviews_final
    


C:\Users\Vinayak11\Anaconda\lib\site-packages\ipykernel\__main__.py:23: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


['wonderful time even great experience goldfish room daughter loved fact valet parking staff chains fabulous staff went make stay oh parking charge would pay garage lot wouldnt help', 'lovely hotel decor friendly front desk staff location its ive staying hotels definitely subtleties establishments provide ive stayed say mostly good hotel monaco decor kimpton hotels always wonderful way colors patterns one cool yet comfortable lobby decorated really nicely tones room nice view romantic feel it better staff friendly helpful without sazerac restaurant us breakfast really great seattle town hotel stay location walked need taxi lots restaurants nearby took good minutes walk pike place market library right across street must see architecture overall nice hotel stay again gave recently stayed wowed experience been hotel monaco gotten another', 'nice hotel expensive parking got good stay hotel arrived late evening took previous reviews valet parking check easy little nonexistent view room room

In [162]:
Model_run(reviews_final,data['Overall'])

C:\Users\Vinayak11\Anaconda\lib\site-packages\ipykernel\__main__.py:23: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


1897
199
{'priors': None}


C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  Deprec

0.375